In [1]:
import itertools
import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [3]:
subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file('vocab_4096')

In [4]:
BATCH_SIZE = 256
BATCHED_ITEM_LENGTH = 256
BUFFER_SIZE = 1024
TYPE=np.int16

checkpoint_dir = './training_checkpoints' # Directory where the checkpoints will be sav
model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ValueError: Input 0 of node sequential_2/lstm_4/AssignVariableOp was passed float from sequential_2/lstm_4/StatefulPartitionedCall/args_1:0 incompatible with expected resource.

Изглежда това няма да сработи.

Ще опитаме нещо различно. Ще пробваме с програмата `tfcompile`. Ще трябва да я свалим и компилираме. Ще последваме инструкциите тук:
<br>
https://github.com/rongjiecomputer/tensorflow-xla-aot-windows/blob/6f204506ecdbd293fc1a7ba6fb3a145724cd6101/README.md
<br>
... и тук:
<br>
https://carlthome.github.io/posts/tfcompile%20/

Ще ни трябва build системата `bazel` версия `2.0.0`. Може лесно да се инсталира през chocolatey:
<br>
https://chocolatey.org/docs/installation

Като допълнителна стъпка се налага да сложа променливата на средата:
<br>
`BAZEL_VC="C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC"`

Освен това по някаква причина трябва да се подаде `--local_ram_resources=2048` на `bazel`. В противен случай компилацията дава грешка `compiler is out of heap space`. Изглежда като поредния бъг в `bazel` -- компютъра има 32 GB RAM.

In [11]:
import tf2xla_pb2

ModuleNotFoundError: No module named 'tf2xla_pb2'

Изглежда ни липсва модул. Ще си го направим. Инсталираме protocol buffers компилатора - `protoc` през Chocolatey. И отново в директорията с git хранилището на tensorflow пускаме:
<br>
```bash
protoc tensorflow/compiler/tf2xla/tf2xla.proto --python_out=.
```

След това копираме получения файл `tf2xla_pb2.py` в директорията с тетрадките. И пробваме отново...

In [5]:
import tf2xla_pb2

In [6]:
config = tf2xla_pb2.Config()

batch_size = 1

for x in model.inputs:
    x.set_shape([batch_size] + list(x.shape)[1:])
    feed = config.feed.add()
    feed.id.node_name = x.op.name
    feed.shape.MergeFrom(x.shape.as_proto())

for x in model.outputs:
    fetch = config.fetch.add()
    fetch.id.node_name = x.op.name

with open('graph.config.pbtxt', 'w') as f:
    f.write(str(config))

In [7]:
!cat graph.config.pbtxt

feed {
  id {
    node_name: "masking_input"
  }
  shape {
    dim {
      size: 1
    }
    dim {
      size: -1
    }
  }
}
fetch {
  id {
    node_name: "dense/Identity"
  }
}


In [51]:
model.save('enwik_model', save_format='tf')

AssertionError: Tried to export a function which references untracked object Tensor("StatefulPartitionedCall/args_2:0", shape=(), dtype=resource).TensorFlow objects (e.g. tf.Variable) captured by functions must be tracked by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

След известно четене и дебъгване (часове... дни) разбираме, че проблемът е `stateful` флагът в LSTM слоевете. Трябва да построим модел без него. Заедно с него направо ще изхвърлим и `recurrent_initializer`.

In [8]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size),
    ])

In [9]:
model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.save('enwik_model', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: enwik_model\assets


Това работи. Вероятно това е и причината за грешката в `TFLiteConverter`. Да се върнем назад и да опитаме отново.

In [56]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ValueError: None is only supported in the 1st dimension. Tensor 'masking_17_input' has invalid shape '[1, None]'.

Получихме различна грешка. Този път е дори разбираема. Да видим какво можем да направим.

In [6]:
def build_predicting_model(vocab_size, embedding_dim, rnn_units):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[1, 1]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size),
    ])

In [60]:
model = build_predicting_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ConverterError: See console for info.
2020-03-26 13:32:03.422726: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
2020-03-26 13:32:06.602298: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-03-26 13:32:06.605061: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-03-26 13:32:06.684594: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.695GHz coreCount: 34 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2020-03-26 13:32:06.684893: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
2020-03-26 13:32:06.716696: I tensorflow/stream_executor/platform/default/d

Защо всичко трябва да е толкова трудно?!

В github issue tracker-а има описан същия проблем:
<br>
https://github.com/tensorflow/tensorflow/issues/33416

И как да бъде заобиколен...

In [61]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()

In [65]:
type(tflite_model)

bytes

In [67]:
os.mkdir('enwik_model_lite')
open('enwik_model_lite/linear.tflite', 'wb').write(tflite_model)

83670768

Над 80 MB... не е много "lite".

In [70]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('enwik_model_lite/linear.tflite', 'wb').write(tflite_model)

ConverterError: See console for info.
2020-03-26 13:56:16.153661: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
2020-03-26 13:56:17.600810: W tensorflow/compiler/mlir/lite/python/graphdef_to_tfl_flatbuffer.cc:89] Ignored output_format.
2020-03-26 13:56:17.600922: W tensorflow/compiler/mlir/lite/python/graphdef_to_tfl_flatbuffer.cc:95] Ignored drop_control_dependency.
2020-03-26 13:56:17.846054: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-03-26 13:56:17.848012: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-03-26 13:56:17.928196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.695GHz coreCount: 34 deviceMemorySize: 8.00G

Нямам добра идея от къде идва този `MemoryError` или как да го оправя. Може би е по-добре да се върнем към подхода с `tfcompile`.

In [11]:
model.save('enwik_model', save_format='tf')

INFO:tensorflow:Assets written to: enwik_model\assets


In [9]:
%pushd D:\dev\work\tensorflow

D:\dev\work\tensorflow


['~\\Desktop\\rnn-enwik-predictor']

In [10]:
def build_predicting_model(vocab_size, embedding_dim, rnn_units):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[1, 1]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size),
    ])

model = build_predicting_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024)
model.load_weights(tf.train.latest_checkpoint(r'C:\Users\joank_000\Desktop\rnn-enwik-predictor\training_checkpoints'))
model.save('graph', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: graph\assets


In [11]:
config = tf2xla_pb2.Config()

batch_size = 1

for x in model.inputs:
    x.set_shape([batch_size] + list(x.shape)[1:])
    feed = config.feed.add()
    feed.id.node_name = x.op.name
    feed.shape.MergeFrom(x.shape.as_proto())

for x in model.outputs:
    fetch = config.fetch.add()
    fetch.id.node_name = x.op.name

with open('graph.config.pbtxt', 'w') as f:
    f.write(str(config))

In [12]:
!cat graph.config.pbtxt

feed {
  id {
    node_name: "masking_1_input"
  }
  shape {
    dim {
      size: 1
    }
    dim {
      size: 1
    }
  }
}
fetch {
  id {
    node_name: "dense_1/Identity"
  }
}


In [13]:
%%writefile BUILD

load('@org_tensorflow//tensorflow/compiler/aot:tfcompile.bzl', 'tf_library')

tf_library(
    name = 'graph',
    config = 'graph.config.pbtxt',
    cpp_class = 'Graph',
    graph = 'graph/saved_model.pb',
)

Overwriting BUILD


In [14]:
!bazel build --show_progress_rate_limit=600 @org_tensorflow//:graph

Starting local Bazel server and connecting to it...
INFO: Options provided by the client:
  Inherited 'common' options: --isatty=0 --terminal_columns=80
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  Inherited 'common' options: --experimental_repo_remote_exec
INFO: Options provided by the client:
  'build' options: --python_path=C:/Users/joank_000/Desktop/rnn-enwik-predictor/env/Scripts/python.exe
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  'build' options: --apple_platform_type=macos --define framework_shared_object=true --define open_source_build=true --java_toolchain=//third_party/toolchains/java:tf_java_toolchain --host_java_toolchain=//third_party/toolchains/java:tf_java_toolchain --define=use_fast_cpp_protos=true --define=allow_oversize_protos=true --spawn_strategy=standalone -c opt --announce_rc --define=grpc_no_ares=true --noincompatible_remove_legacy_whole_archive --noincompatible_prohibit_aapt1 --enable_platform_s

INFO: Analyzed target //:graph (176 packages loaded, 16509 targets configured).
INFO: Found 1 target...
INFO: Deleting stale sandbox base C:/users/joank_000/_bazel_joank_000/uvqhxzj5/sandbox
[0 / 213] [Prepa] BazelWorkspaceStatusAction stable-status.txt
ERROR: C:/users/joank_000/_bazel_joank_000/uvqhxzj5/external/com_google_absl/absl/strings/BUILD.bazel:30:1: C++ compilation of rule '@com_google_absl//absl/strings:strings' failed (Exit 1)
 
The target you are compiling requires Visual C++ build tools. 
Bazel couldn't find a valid Visual C++ build tools installation on your machine. 
 
Visual C++ build tools seems to be installed at C:\Program Files (x86)\Microsoft Visual Studio 14.0\\VC 
But Bazel can't find the following tools: 
    VCVARSALL.BAT, cl.exe, link.exe, lib.exe, ml64.exe 
 
Please check your installation following https://docs.bazel.build/versions/master/windows.html#using 
 
Target //:graph failed to build
INFO: Elapsed time: 399.001s, Critical Path: 0.16s
INFO: 0 process

Изглежда пак ни липсва `BAZEL_VC`...

In [19]:
os.environ['BAZEL_VC'] = r'C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC'

In [25]:
!bazel build --show_progress_rate_limit=600 --local_resources=2048,.5,1.0 @org_tensorflow//:graph

C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\xmemory(671): warning C4267: 'argument': conversion from 'size_t' to 'int', possible loss of data
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\vector(687): note: see reference to function template instantiation 'void std::_Default_allocator_traits<_Alloc>::construct<_Ty,tensorflow::grappler::utils::GraphView*,const int&,unsigned __int64>(_Alloc &,_Objty *const ,tensorflow::grappler::utils::GraphView *&&,const int &,unsigned __int64 &&)' being compiled
        with
        [
            _Alloc=std::allocator<tensorflow::grappler::utils::FaninView>,
            _Ty=tensorflow::grappler::utils::FaninView,
            _Objty=tensorflow::grappler::utils::FaninView
        ]
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\vector(687): note: see reference to function template instantiation 'void std::

INFO: Options provided by the client:
  Inherited 'common' options: --isatty=0 --terminal_columns=80
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  Inherited 'common' options: --experimental_repo_remote_exec
INFO: Options provided by the client:
  'build' options: --python_path=C:/Users/joank_000/Desktop/rnn-enwik-predictor/env/Scripts/python.exe
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  'build' options: --apple_platform_type=macos --define framework_shared_object=true --define open_source_build=true --java_toolchain=//third_party/toolchains/java:tf_java_toolchain --host_java_toolchain=//third_party/toolchains/java:tf_java_toolchain --define=use_fast_cpp_protos=true --define=allow_oversize_protos=true --spawn_strategy=standalone -c opt --announce_rc --define=grpc_no_ares=true --noincompatible_remove_legacy_whole_archive --noincompatible_prohibit_aapt1 --enable_platform_specific_config --config=v2
INFO: Reading rc options 

C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\utility(277): warning C4244: '=': conversion from '_Ty' to '_Ty1', possible loss of data
        with
        [
            _Ty=std::chrono::system_clock::rep
        ]
        and
        [
            _Ty1=int
        ]
tensorflow/core/grappler/costs/virtual_scheduler.cc(846): note: see reference to function template instantiation 'std::pair<int,bool> &std::pair<int,bool>::operator =<__int64,bool,0>(std::pair<__int64,bool> &&) noexcept' being compiled
tensorflow/core/grappler/costs/virtual_scheduler.cc(844): note: see reference to function template instantiation 'std::pair<int,bool> &std::pair<int,bool>::operator =<__int64,bool,0>(std::pair<__int64,bool> &&) noexcept' being compiled
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\type_traits(1636): warning C4244: 'return': conversion from 'double' to '_Rx', possible loss of data
        with


INFO: Analyzed target //:graph (0 packages loaded, 0 targets configured).
INFO: Found 1 target...
[0 / 112] [Prepa] BazelWorkspaceStatusAction stable-status.txt
INFO: From Compiling external/com_google_absl/absl/base/dynamic_annotations.cc:
cl : Command line warning D9025 : overriding '/w' with '/W3'
INFO: From Compiling external/com_google_absl/absl/numeric/int128.cc:
cl : Command line warning D9025 : overriding '/w' with '/W3'
INFO: From Compiling external/com_google_absl/absl/base/internal/spinlock_wait.cc:
cl : Command line warning D9025 : overriding '/w' with '/W3'
INFO: From Compiling external/com_google_absl/absl/base/internal/unscaledcycleclock.cc:
cl : Command line warning D9025 : overriding '/w' with '/W3'
INFO: From Compiling external/com_google_absl/absl/base/internal/throw_delegate.cc:
cl : Command line warning D9025 : overriding '/w' with '/W3'
INFO: From Compiling external/com_google_absl/absl/base/internal/thread_identity.cc:
cl : Command line warning D9025 : overriding

            ReturnT=tensorflow::uint32
        ]
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\functional(807): note: see reference to class template instantiation 'std::_Func_impl_no_alloc<xla::HloEvaluatorTypedVisitor<tensorflow::uint32,ReturnT>::HandleExp::<lambda_8e271de2834890b0dc7c4031797902f7>,_Ret,ReturnT>' being compiled
        with
        [
            ReturnT=tensorflow::uint32,
            _Ret=tensorflow::uint32
        ]
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\functional(1019): note: see reference to variable template 'const bool _Is_large<std::_Func_impl_no_alloc<<lambda_8e271de2834890b0dc7c4031797902f7>,unsigned int,unsigned int> >' being compiled
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\functional(1155): note: see reference to function template instantiation 'void std::_Func_class<_Ret,ReturnT>::_Reset<xla::H

            ReturnT=tensorflow::int32,
            _Ret=tensorflow::int32
        ]
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\functional(1019): note: see reference to variable template 'const bool _Is_large<std::_Func_impl_no_alloc<<lambda_7773ba64924f32ec3e63591dfd8cb42b>,int,int> >' being compiled
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.25.28610\include\functional(1155): note: see reference to function template instantiation 'void std::_Func_class<_Ret,ReturnT>::_Reset<xla::HloEvaluatorTypedVisitor<ReturnT,ReturnT>::HandleExp::<lambda_7773ba64924f32ec3e63591dfd8cb42b>>(_Fx &&)' being compiled
        with
        [
            _Ret=tensorflow::int32,
            ReturnT=tensorflow::int32,
            _Fx=xla::HloEvaluatorTypedVisitor<tensorflow::int32,tensorflow::int32>::HandleExp::<lambda_7773ba64924f32ec3e63591dfd8cb42b>
        ]
C:\Program Files (x86)\Microsoft Visual Studio\2019\Communit

        ]
.\tensorflow/compiler/xla/service/hlo_evaluator_typed_visitor.h(2365): note: see reference to function template instantiation 'std::function<ReturnT (ReturnT)>::function<xla::HloEvaluatorTypedVisitor<ReturnT,ReturnT>::HandlePopulationCount::<lambda_349add2dd37cd2470990cd267b3c29bb>,0>(_Fx)' being compiled
        with
        [
            ReturnT=tensorflow::int32,
            _Fx=xla::HloEvaluatorTypedVisitor<tensorflow::int32,tensorflow::int32>::HandlePopulationCount::<lambda_349add2dd37cd2470990cd267b3c29bb>
        ]
.\tensorflow/compiler/xla/service/hlo_evaluator_typed_visitor.h(2360): note: see reference to function template instantiation 'std::function<ReturnT (ReturnT)>::function<xla::HloEvaluatorTypedVisitor<ReturnT,ReturnT>::HandlePopulationCount::<lambda_349add2dd37cd2470990cd267b3c29bb>,0>(_Fx)' being compiled
        with
        [
            ReturnT=tensorflow::int32,
            _Fx=xla::HloEvaluatorTypedVisitor<tensorflow::int32,tensorflow::int32>::HandlePop

        [
            _Ret=tensorflow::int8,
            ReturnT=tensorflow::int8,
            _Fx=xla::HloEvaluatorTypedVisitor<tensorflow::int8,tensorflow::int8>::HandleExp::<lambda_14d1e92aa8cd35ab3474b8c7cf815b05>
        ]
.\tensorflow/compiler/xla/service/hlo_evaluator_typed_visitor.h(299): note: see reference to function template instantiation 'std::function<ReturnT (ReturnT)>::function<xla::HloEvaluatorTypedVisitor<ReturnT,ReturnT>::HandleExp::<lambda_14d1e92aa8cd35ab3474b8c7cf815b05>,0>(_Fx)' being compiled
        with
        [
            ReturnT=tensorflow::int8,
            _Fx=xla::HloEvaluatorTypedVisitor<tensorflow::int8,tensorflow::int8>::HandleExp::<lambda_14d1e92aa8cd35ab3474b8c7cf815b05>
        ]
.\tensorflow/compiler/xla/service/hlo_evaluator_typed_visitor.h(296): note: see reference to function template instantiation 'std::function<ReturnT (ReturnT)>::function<xla::HloEvaluatorTypedVisitor<ReturnT,ReturnT>::HandleExp::<lambda_14d1e92aa8cd35ab3474b8c7cf815b05>,0>

In [26]:
!bazel build --show_progress_rate_limit=600 --local_resources=2048,.5,1.0 @org_tensorflow//:graph

INFO: Options provided by the client:
  Inherited 'common' options: --isatty=0 --terminal_columns=80
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  Inherited 'common' options: --experimental_repo_remote_exec
INFO: Options provided by the client:
  'build' options: --python_path=C:/Users/joank_000/Desktop/rnn-enwik-predictor/env/Scripts/python.exe
INFO: Reading rc options for 'build' from d:\dev\work\tensorflow\.bazelrc:
  'build' options: --apple_platform_type=macos --define framework_shared_object=true --define open_source_build=true --java_toolchain=//third_party/toolchains/java:tf_java_toolchain --host_java_toolchain=//third_party/toolchains/java:tf_java_toolchain --define=use_fast_cpp_protos=true --define=allow_oversize_protos=true --spawn_strategy=standalone -c opt --announce_rc --define=grpc_no_ares=true --noincompatible_remove_legacy_whole_archive --noincompatible_prohibit_aapt1 --enable_platform_specific_config --config=v2
INFO: Reading rc options 

INFO: Analyzed target //:graph (0 packages loaded, 0 targets configured).
INFO: Found 1 target...
[0 / 3] [Prepa] Executing genrule //:gen_graph
ERROR: D:/dev/work/tensorflow/BUILD:4:1: Executing genrule //:gen_graph failed (Exit 127)
2020-03-31 03:44:47.830611: F tensorflow/compiler/aot/tfcompile_main.cc:91] Non-OK-status: status status: Data loss: Can't parse graph/saved_model.pb as binary proto
Target //:graph failed to build
INFO: Elapsed time: 4.707s, Critical Path: 0.14s
INFO: 0 processes.
FAILED: Build did NOT complete successfully
FAILED: Build did NOT complete successfully


In [182]:
%popd

UsageError: %popd on empty stack


In [196]:
checkpoint_dir

'./training_checkpoints'